In [1]:
# if importing g2o does not work, uncomment the line above
#!python -m pip install g2o-python
import g2o

In [2]:
import numpy as np

class GraphSLAM2D:
    def __init__(self, verbose=False) -> None:
        '''
        GraphSLAM in 2D with G2O
        '''
        self.optimizer = g2o.SparseOptimizer()
        self.solver = g2o.BlockSolverX(g2o.LinearSolverDenseX())
        self.algorithm = g2o.OptimizationAlgorithmLevenberg(self.solver)
        self.optimizer.set_algorithm(self.algorithm)

        self.vertex_count = 0
        self.edge_count = 0
        self.verbose = verbose

    def vertex_pose(self, id):
        '''
        Get position of vertex by id
        '''
        return self.optimizer.vertex(id).estimate()

    def vertex(self, id):
        '''
        Get vertex by id
        '''
        return self.optimizer.vertex(id)

    def edge(self, id):
        '''
        Get edge by id
        '''
        return self.optimizer.edge(id)

    def add_fixed_pose(self, pose, vertex_id=None):
        '''
        Add fixed pose to the graph
        '''
        v_se2 = g2o.VertexSE2()
        if vertex_id is None:
            vertex_id = self.vertex_count
        v_se2.set_id(vertex_id)
        if self.verbose:
            print("Adding fixed pose vertex with ID", vertex_id)
        v_se2.set_estimate(pose)
        v_se2.set_fixed(True)
        self.optimizer.add_vertex(v_se2)
        self.vertex_count += 1

    def add_odometry(self, northings, eastings, heading, information):
        '''
        Add odometry to the graph
        '''
        # Find the last pose vertex id
        vertices = self.optimizer.vertices()
        if len(vertices) > 0:
            last_id = [v for v in vertices if type(vertices[v]) == g2o.VertexSE2][0]
            print("Last id is", last_id)
        else:
            raise ValueError("There is no previous pose, have you forgot to add a fixed initial pose?")
        v_se2 = g2o.VertexSE2()
        if self.verbose:
            print("Adding pose vertex", self.vertex_count)
        v_se2.set_id(self.vertex_count)
        pose = g2o.SE2(northings, eastings, heading)
        v_se2.set_estimate(pose)
        self.optimizer.add_vertex(v_se2)
        # add edge
        e_se2 = g2o.EdgeSE2()
        e_se2.set_vertex(0, self.vertex(last_id))
        e_se2.set_vertex(1, self.vertex(self.vertex_count))
        e_se2.set_measurement(pose)
        e_se2.set_information(information)
        self.optimizer.add_edge(e_se2)
        self.vertex_count += 1
        self.edge_count += 1
        if self.verbose:
            print("Adding SE2 edge between", last_id, self.vertex_count-1)

    def add_landmark(self, x, y, information, pose_id, landmark_id=None):
        '''
        Add landmark to the graph
        '''
        relative_measurement = np.array([x, y])
        
        # Check that the pose_id is of type VertexSE2
        if type(self.optimizer.vertex(pose_id)) != g2o.VertexSE2:
            raise ValueError("The pose_id that you have provided does not correspond to a VertexSE2")
        
        trans0 = self.optimizer.vertex(pose_id).estimate()
        measurement = trans0 * relative_measurement
        
        print(relative_measurement, measurement)
        
        if landmark_id is None:
            landmark_id = self.vertex_count
            v_pointxy = g2o.VertexPointXY()
            v_pointxy.set_estimate(measurement)
            v_pointxy.set_id(landmark_id)
            if self.verbose:
                print("Adding landmark vertex", landmark_id)
            self.optimizer.add_vertex(v_pointxy)
            self.vertex_count += 1
        # add edge
        e_pointxy = g2o.EdgeSE2PointXY()
        e_pointxy.set_vertex(0, self.vertex(pose_id))
        e_pointxy.set_vertex(1, self.vertex(landmark_id))
        self.edge_count += 1
        e_pointxy.set_measurement(relative_measurement)
        e_pointxy.set_information(information)
        self.optimizer.add_edge(e_pointxy)
        if self.verbose:
            print("Adding landmark edge between", pose_id, landmark_id)

    def optimize(self, iterations=10, verbose=None):
        '''
        Optimize the graph
        '''
        self.optimizer.initialize_optimization()
        if verbose is None:
            verbose = self.verbose
        self.optimizer.set_verbose(verbose)
        self.optimizer.optimize(iterations)
        return self.optimizer.chi2()

In [3]:
graph_slam = GraphSLAM2D(verbose=True)

# Add fixed pose ID #0
graph_slam.add_fixed_pose(g2o.SE2())

# Add a landmark #1
landmark_x = 0
landmark_y = 1
graph_slam.add_landmark(landmark_x, landmark_y, np.eye(2), pose_id=0)

# Add odometry #2
graph_slam.add_odometry(1, 0, 0, 0.1*np.eye(3))
 
# Add a landmark #3
landmark_x = 0
landmark_y = 1
graph_slam.add_landmark(landmark_x, landmark_y, np.eye(2), pose_id=2)

# Add another odometry #4
graph_slam.add_odometry(2, 0, 0, 0.1*np.eye(3))

# Add a new landmark #5
landmark_x = 0
landmark_y = 1
graph_slam.add_landmark(landmark_x, landmark_y, np.eye(2), pose_id=4)

# Add a new landmark relationship between ID #2 and ID #5
landmark_x = 1
landmark_y = 1
graph_slam.add_landmark(landmark_x, landmark_y, np.eye(2), pose_id=2, landmark_id=5)

# Optimize
graph_slam.optimize(10, verbose=True)

Adding fixed pose vertex with ID 0
[0 1] [0. 1.]
Adding landmark vertex 1
Adding landmark edge between 0 1
Last id is 0
Adding pose vertex 2
Adding SE2 edge between 0 2
[0 1] [1. 1.]
Adding landmark vertex 3
Adding landmark edge between 2 3
Last id is 2
Adding pose vertex 4
Adding SE2 edge between 2 4
[0 1] [2. 1.]
Adding landmark vertex 5
Adding landmark edge between 4 5
[1 1] [2. 1.]
Adding landmark edge between 2 5


iteration= 0	 chi2= 0.057383	 time= 3.15001e-05	 cumTime= 3.15001e-05	 edges= 6	 schur= 0	 lambda= 0.000022	 levenbergIter= 1
iteration= 1	 chi2= 0.047582	 time= 9.31323e-06	 cumTime= 4.08133e-05	 edges= 6	 schur= 0	 lambda= 0.000009	 levenbergIter= 1
iteration= 2	 chi2= 0.047501	 time= 5.82077e-06	 cumTime= 4.66341e-05	 edges= 6	 schur= 0	 lambda= 0.000006	 levenbergIter= 1
iteration= 3	 chi2= 0.047498	 time= 5.07897e-06	 cumTime= 5.17131e-05	 edges= 6	 schur= 0	 lambda= 0.000004	 levenbergIter= 1
iteration= 4	 chi2= 0.047498	 time= 4.4941e-06	 cumTime= 5.62072e-05	 edges= 6	 schur= 0	 lambda= 0.000003	 levenbergIter= 1
iteration= 5	 chi2= 0.047498	 time= 4.45591e-06	 cumTime= 6.06631e-05	 edges= 6	 schur= 0	 lambda= 0.000002	 levenbergIter= 1
iteration= 6	 chi2= 0.047498	 time= 6.09783e-06	 cumTime= 6.67609e-05	 edges= 6	 schur= 0	 lambda= 0.000001	 levenbergIter= 1
iteration= 7	 chi2= 0.047498	 time= 4.57885e-06	 cumTime= 7.13398e-05	 edges= 6	 schur= 0	 lambda= 0.000001	 levenbergI

0.04749837609735881

In [6]:
from g2o import SparseBlockMatrixX

vertices = graph_slam.optimizer.vertices()
print("len(vertices):", len(vertices))

cov_vertices = []
covariances = SparseBlockMatrixX()
for vertex_idx in vertices:
    v = vertices[vertex_idx]
    if type(v) == g2o.VertexSE2 and not v.fixed():
        cov_vertices.append((v.hessian_index(), v.hessian_index()))
print("Sparse pattern contains", len(cov_vertices), "blocks")
print(cov_vertices)

len(vertices): 6
Sparse pattern contains 2 blocks
[(3, 3), (1, 1)]


In [11]:
print("Computing covariance matrix...")
m = graph_slam.optimizer.compute_marginals(covariances, cov_vertices)
print("Done,", m)

print(covariances.rows())
print(covariances.cols())

Computing covariance matrix...
Done, False
0
0


In [18]:
print("Covariance matrix:")
for vertex_idx in vertices:
    v = vertices[vertex_idx]
    if type(v) == g2o.VertexSE2 and not v.fixed():
        print(" Working on vertex idx:", vertex_idx)
        print("  Hessian idx is:", v.hessian_index())
        if covariances.rows() > v.hessian_index() and covariances.cols() > v.hessian_index():
            covariance = covariances.block(v.hessian_index(), v.hessian_index());
            print("  Covariance is:", covariance)
        else:
            print("  ERROR: The covariance matrix is not big enough. Something is wrong.")

Covariance matrix:
 Working on vertex idx: 4
  Hessian idx is: 3
  ERROR: The covariance matrix is not big enough. Something is wrong.
 Working on vertex idx: 2
  Hessian idx is: 1
  ERROR: The covariance matrix is not big enough. Something is wrong.
